In [1]:
import mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def layer(intput,weight_shape,bias_shape):
    weight_stddev = (2.0/weight_shape[0])**0.5
    w_init=tf.random_normal_initializer(stddev=weight_stddev)
    bias_init = tf.constant_initializer(value=0)
    W= tf.get_variable('W',weight_shape,initializer=w_init)
    b= tf.get_variable('b',bias_shape, initializer=bias_init)
    return tf.nn.relu(tf.matmul(input,W)+b)
def inference(x):
    with tf.variable_scope("hidden_1"):
        hidden_1 = layer(x,[784,256],[256])
    with tf.variable_scope("hidden_2"):
        hidden_2 = layer(hidden_1,[256,256],[256])
    with tf.variable_scope("output"):
        output = layer(hidden_2,[256,10],[10])
    return output
def loss(output,y):
    """softmax loss"""
    xenotropy = tf.nn.softmax_cross_entropy_with_logits(output,y)
    loss=tf.reduce_mean(xenotropy)
    return loss

In [3]:
# sudo pip install python-mnist
tf.reset_default_graph()
##### PARAMETERS ####
learning_rate= 0.01
training_epochs=300
batch_size=100
display_step=30
#####################
with tf.Graph().as_default():
    # mnist data image shape 28*28=784
    x=tf.placeholder("float",[None,784])
    # 0-9 digits recognition >=10 classes
    y=tf.placeholder("float",[None,10])
    
    output=inference(x)
    cost = loss(output,y)
    global_step= tf.Variable(0,name='global_step',trainable=False)
    train_op = training(cost,global_step)
    eval_op = evaluate(output,y)
    summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    sess=tf.Session()
    summary_writer=tf.summary.FileWriter("logistic_logs/",graph_def=sess.graph_def)
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    #### TRAINING CYCLE ####
    for epoch in range(training_epochs):
        avg_cost=0.
        total_batch=int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            mbatch_x,mbatch_y= mnist.train.next_batch(batch_size)
            # Fit training with batch
            feed_dict = {x:mbatch_x,y:mbatch_y}
            sess.run(train_op,feed_dict=feed_dict)
            # Compute average loss
            minibatch_cost = sess.run(cost,feed_dict=feed_dict)
            avg_cost += minibatch_cost/total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
                val_feed_dict = {
                            x: mnist.validation.images,
                            y: mnist.validation.labels
                }
                accuracy = sess.run(eval_op,feed_dict=val_feed_dict)
                print "Validation Error:", (1-accuracy)
                summary_str = sess.run(summary_op,feed_dict=feed_dict)
                summary_writer.add_summary(summary_str,sess.run(global_step))
                saver.save(sess,"feed_forward_logs/model-checkpoint",global_step=global_step)
                summary_writer.flush()
    print "---OPTIMIZATION COMPLETE---"
    test_feed_dict= {
        x:mnist.test.images,
        y:mnist.test.labels
    }
    accuracy = sess.run(eval_op,feed_dict=test_feed_dict)
    print "Test Accuracy: {}".format(accuracy)
    ########################

TypeError: Expected binary or unicode string, got <function <lambda> at 0x7fcd946348c0>